# 5.2.1 OpenCV DNN 模組整合 (OpenCV DNN Module Integration)

**WBS 5.2.1**: 深度學習模型整合與推理

本模組深入探討 OpenCV DNN 模組的使用，學習如何載入並運行來自不同深度學習框架的預訓練模型。

## 學習目標
- 理解 OpenCV DNN 模組的架構與優勢
- 掌握多種深度學習框架模型的載入方法
- 實作人臉檢測、物體檢測、圖像分類、語義分割
- 學習模型推理優化技巧
- 比較不同 backend 和 target 的性能差異

## 前置知識
- 深度學習基礎概念
- OpenCV 圖像處理 (Stage 2 & 3)
- Python 與 NumPy
- 傳統機器學習方法 (WBS 5.1)

## 課程大綱
1. OpenCV DNN 模組簡介 (5%)
2. 支持的深度學習框架 (10%)
3. 載入預訓練模型 (15%)
4. 人臉檢測 (15%)
5. 物體檢測 (20%)
6. 圖像分類 (10%)
7. 語義分割 (10%)
8. 性能優化 (10%)
9. 實戰練習 (3%)
10. 總結與延伸 (2%)

In [ ]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
import urllib.request
from collections import defaultdict

# Configure matplotlib for Chinese display
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (14, 8)

# Disable warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")
print(f"OpenCV version: {cv2.__version__}")
print(f"OpenCV DNN module available: {hasattr(cv2, 'dnn')}")

# Check available backends and targets
print("\nAvailable DNN backends:")
backends = [
    ("DEFAULT", cv2.dnn.DNN_BACKEND_DEFAULT),
    ("HALIDE", cv2.dnn.DNN_BACKEND_HALIDE),
    ("INFERENCE_ENGINE", cv2.dnn.DNN_BACKEND_INFERENCE_ENGINE),
    ("OPENCV", cv2.dnn.DNN_BACKEND_OPENCV),
    ("VKCOM", cv2.dnn.DNN_BACKEND_VKCOM),
    ("CUDA", cv2.dnn.DNN_BACKEND_CUDA)
]

for name, backend in backends:
    print(f"  - {name}: {backend}")

print("\nAvailable DNN targets:")
targets = [
    ("CPU", cv2.dnn.DNN_TARGET_CPU),
    ("OPENCL", cv2.dnn.DNN_TARGET_OPENCL),
    ("OPENCL_FP16", cv2.dnn.DNN_TARGET_OPENCL_FP16),
    ("MYRIAD", cv2.dnn.DNN_TARGET_MYRIAD),
    ("CUDA", cv2.dnn.DNN_TARGET_CUDA),
    ("CUDA_FP16", cv2.dnn.DNN_TARGET_CUDA_FP16)
]

for name, target in targets:
    print(f"  - {name}: {target}")

## 1. OpenCV DNN 模組簡介 (Introduction) - 5%

### 什麼是 OpenCV DNN 模組？

**OpenCV DNN (Deep Neural Networks) 模組**是 OpenCV 3.3 版本引入的深度學習推理引擎，讓開發者能夠直接在 OpenCV 中運行深度學習模型。

### 核心優勢

1. **統一接口**: 單一 API 支持多種深度學習框架
2. **零依賴**: 不需要安裝 TensorFlow、PyTorch 等框架
3. **跨平台**: 支援 Windows、Linux、macOS、Android、iOS
4. **高效率**: 針對推理優化，支援多種硬體加速
5. **輕量級**: 模型推理無需完整深度學習框架

### OpenCV DNN vs 深度學習框架

| 特性 | OpenCV DNN | TensorFlow/PyTorch |
|------|-----------|-------------------|
| **用途** | 推理 (Inference) | 訓練 + 推理 |
| **安裝** | 內建於 OpenCV | 需額外安裝 |
| **大小** | 輕量 (~50MB) | 重量 (>500MB) |
| **速度** | 優化過的推理 | 通用框架 |
| **部署** | 簡單 | 複雜 |
| **GPU支援** | CUDA, OpenCL | CUDA, ROCm |

### 適用場景

✅ **適合使用 OpenCV DNN**:
- 僅需模型推理，不需訓練
- 希望減少依賴和部署複雜度
- 嵌入式或移動設備部署
- 與 OpenCV 視覺管線整合

❌ **不適合使用 OpenCV DNN**:
- 需要訓練模型
- 需要最新的模型架構
- 需要自定義層或複雜操作

### DNN 模組架構

```
Model Loading → Preprocessing → Forward Pass → Post-processing
     ↓              ↓              ↓               ↓
readNet()    blobFromImage()   net.forward()   NMS, decode
```

## 2. 支持的深度學習框架 (Supported Frameworks) - 10%

### 支援的框架與格式

OpenCV DNN 模組支援多種深度學習框架導出的模型格式。

#### 1. Caffe
- **文件**: `.caffemodel` (weights) + `.prototxt` (architecture)
- **載入**: `cv2.dnn.readNetFromCaffe(prototxt, caffemodel)`
- **優點**: 早期主流框架，模型豐富
- **應用**: 人臉檢測、影像分類、語義分割

```python
net = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'model.caffemodel')
```

#### 2. TensorFlow
- **文件**: `.pb` (frozen graph) 或 SavedModel 格式
- **載入**: `cv2.dnn.readNetFromTensorflow(model, config)`
- **優點**: Google 支援，模型最多
- **應用**: 物體檢測 (SSD, Faster R-CNN)、影像分類

```python
net = cv2.dnn.readNetFromTensorflow('frozen_inference_graph.pb', 'config.pbtxt')
```

#### 3. PyTorch
- **文件**: `.onnx` (需先轉換)
- **載入**: `cv2.dnn.readNetFromONNX(onnx_file)`
- **優點**: 研究界主流，模型創新快
- **應用**: 最新研究模型

```python
# PyTorch model must be exported to ONNX first
# torch.onnx.export(model, dummy_input, "model.onnx")
net = cv2.dnn.readNetFromONNX('model.onnx')
```

#### 4. ONNX
- **文件**: `.onnx` (open format)
- **載入**: `cv2.dnn.readNetFromONNX(onnx_file)`
- **優點**: 跨框架標準格式
- **應用**: 框架間模型轉換

```python
net = cv2.dnn.readNetFromONNX('model.onnx')
```

#### 5. Darknet (YOLO)
- **文件**: `.weights` (weights) + `.cfg` (config)
- **載入**: `cv2.dnn.readNetFromDarknet(cfg, weights)`
- **優點**: YOLO 系列專用
- **應用**: 實時物體檢測

```python
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
```

#### 6. Torch (Legacy)
- **文件**: `.t7` (Lua Torch)
- **載入**: `cv2.dnn.readNetFromTorch(model)`
- **優點**: 老舊模型相容
- **應用**: 風格轉換等舊模型

### 通用載入函數

```python
# Automatically detect framework
net = cv2.dnn.readNet('model_file', 'config_file')
```

In [ ]:
# Framework comparison table
framework_info = {
    "Framework": ["Caffe", "TensorFlow", "PyTorch (ONNX)", "ONNX", "Darknet (YOLO)", "Torch (Legacy)"],
    "File Format": [
        ".caffemodel + .prototxt",
        ".pb + .pbtxt",
        ".onnx",
        ".onnx",
        ".weights + .cfg",
        ".t7"
    ],
    "Load Function": [
        "readNetFromCaffe()",
        "readNetFromTensorflow()",
        "readNetFromONNX()",
        "readNetFromONNX()",
        "readNetFromDarknet()",
        "readNetFromTorch()"
    ],
    "Typical Use": [
        "Face detection, Classification",
        "Object detection (SSD)",
        "Latest research models",
        "Cross-framework models",
        "YOLO object detection",
        "Style transfer (old)"
    ]
}

print("\nSupported Deep Learning Frameworks:")
print("=" * 120)
print(f"{'Framework':<20} {'File Format':<30} {'Load Function':<30} {'Typical Use':<40}")
print("=" * 120)

for i in range(len(framework_info["Framework"])):
    print(f"{framework_info['Framework'][i]:<20} "
          f"{framework_info['File Format'][i]:<30} "
          f"{framework_info['Load Function'][i]:<30} "
          f"{framework_info['Typical Use'][i]:<40}")
print("=" * 120)

## 3. 載入預訓練模型 (Loading Models) - 15%

### 模型下載與準備

在使用 OpenCV DNN 之前，我們需要下載預訓練模型。

### 常用模型資源

1. **OpenCV Model Zoo**: https://github.com/opencv/opencv/wiki/TensorFlow-Object-Detection-API
2. **Caffe Model Zoo**: https://github.com/BVLC/caffe/wiki/Model-Zoo
3. **ONNX Model Zoo**: https://github.com/onnx/models
4. **Darknet (YOLO)**: https://pjreddie.com/darknet/yolo/
5. **TensorFlow Hub**: https://www.tensorflow.org/hub

### Model Download Helper

In [ ]:
# Model download utility
class ModelDownloader:
    """
    Utility class for downloading pre-trained models
    """
    
    def __init__(self, base_path='../assets/models'):
        self.base_path = Path(base_path)
        self.base_path.mkdir(parents=True, exist_ok=True)
    
    def download(self, url, filename, force=False):
        """
        Download file from URL
        
        Parameters:
        -----------
        url : str
            URL to download from
        filename : str
            Local filename to save
        force : bool
            Force re-download if file exists
        """
        filepath = self.base_path / filename
        
        if filepath.exists() and not force:
            print(f"✓ {filename} already exists")
            return str(filepath)
        
        print(f"Downloading {filename}...")
        try:
            urllib.request.urlretrieve(url, str(filepath))
            print(f"✓ Downloaded {filename} ({filepath.stat().st_size / 1024 / 1024:.2f} MB)")
            return str(filepath)
        except Exception as e:
            print(f"✗ Failed to download {filename}: {e}")
            return None

# Initialize downloader
downloader = ModelDownloader()

# Model URLs (these are examples - actual URLs may change)
MODEL_URLS = {
    # Face detection (Caffe)
    'face_detector_prototxt': 'https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt',
    'face_detector_model': 'https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel',
    
    # MobileNet-SSD (Caffe)
    'mobilenet_prototxt': 'https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/deploy.prototxt',
    'mobilenet_model': 'https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/mobilenet_iter_73000.caffemodel',
}

print("Available models for download:")
for key, url in MODEL_URLS.items():
    print(f"  - {key}: {url[:80]}...")

### 模型載入示例

讓我們示範如何載入不同類型的模型。

In [ ]:
def load_model_safe(model_type, model_path, config_path=None):
    """
    Safely load DNN model with error handling
    
    Parameters:
    -----------
    model_type : str
        Type of model ('caffe', 'tensorflow', 'onnx', 'darknet', 'torch')
    model_path : str
        Path to model weights file
    config_path : str, optional
        Path to model configuration file
        
    Returns:
    --------
    net : cv2.dnn.Net or None
        Loaded network or None if failed
    """
    try:
        if model_type.lower() == 'caffe':
            if config_path is None:
                raise ValueError("Caffe requires config_path (prototxt)")
            net = cv2.dnn.readNetFromCaffe(config_path, model_path)
            
        elif model_type.lower() == 'tensorflow':
            if config_path:
                net = cv2.dnn.readNetFromTensorflow(model_path, config_path)
            else:
                net = cv2.dnn.readNetFromTensorflow(model_path)
                
        elif model_type.lower() == 'onnx':
            net = cv2.dnn.readNetFromONNX(model_path)
            
        elif model_type.lower() == 'darknet':
            if config_path is None:
                raise ValueError("Darknet requires config_path (.cfg)")
            net = cv2.dnn.readNetFromDarknet(config_path, model_path)
            
        elif model_type.lower() == 'torch':
            net = cv2.dnn.readNetFromTorch(model_path)
            
        else:
            raise ValueError(f"Unsupported model type: {model_type}")
        
        print(f"✓ Successfully loaded {model_type} model from {Path(model_path).name}")
        
        # Print network information
        layer_names = net.getLayerNames()
        print(f"  Total layers: {len(layer_names)}")
        print(f"  Input layers: {net.getUnconnectedOutLayers()}")
        
        return net
        
    except Exception as e:
        print(f"✗ Failed to load {model_type} model: {e}")
        return None

# Example: Load model using generic readNet (auto-detect format)
print("\nGeneric model loading (auto-detect):")
print("cv2.dnn.readNet() automatically detects the framework based on file extensions")
print("Example: net = cv2.dnn.readNet('model.caffemodel', 'deploy.prototxt')")

### 檢查模型結構

In [ ]:
def inspect_network(net, verbose=False):
    """
    Inspect DNN network structure
    
    Parameters:
    -----------
    net : cv2.dnn.Net
        Loaded network
    verbose : bool
        Print detailed layer information
    """
    if net is None:
        print("Network is None")
        return
    
    # Get layer information
    layer_names = net.getLayerNames()
    layer_ids = net.getUnconnectedOutLayers()
    
    print(f"\nNetwork Structure:")
    print(f"  Total layers: {len(layer_names)}")
    print(f"  Output layers: {len(layer_ids)}")
    
    # Get output layer names
    output_layers = [layer_names[i - 1] for i in layer_ids]
    print(f"  Output layer names: {output_layers}")
    
    if verbose and len(layer_names) < 50:
        print(f"\n  All layers:")
        for i, name in enumerate(layer_names[:20], 1):
            print(f"    {i:3d}. {name}")
        if len(layer_names) > 20:
            print(f"    ... ({len(layer_names) - 20} more layers)")

print("Network inspection utility defined.")
print("Use inspect_network(net, verbose=True) to see detailed layer information.")

## 4. 人臉檢測 (Face Detection with DNN) - 15%

### 使用 ResNet-based SSD 進行人臉檢測

OpenCV 提供基於 ResNet-10 的 SSD 人臉檢測模型，相比 Haar Cascade 有更高的準確度。

### 模型特點
- **架構**: Single Shot Detector (SSD) with ResNet-10 backbone
- **輸入**: 300×300 RGB image
- **輸出**: Face bounding boxes with confidence scores
- **優勢**: 比 Haar Cascade 更準確，處理側臉和遮擋更好

In [ ]:
# Setup face detection model paths
FACE_MODEL_DIR = Path('../assets/models/face_detection')
FACE_MODEL_DIR.mkdir(parents=True, exist_ok=True)

face_prototxt = FACE_MODEL_DIR / 'deploy.prototxt'
face_model = FACE_MODEL_DIR / 'res10_300x300_ssd_iter_140000_fp16.caffemodel'

# Check if models exist
if face_prototxt.exists() and face_model.exists():
    print("✓ Face detection models found")
    FACE_MODEL_AVAILABLE = True
else:
    print("⚠ Face detection models not found")
    print("Please download:")
    print("  1. deploy.prototxt")
    print("  2. res10_300x300_ssd_iter_140000_fp16.caffemodel")
    print(f"  Save to: {FACE_MODEL_DIR}")
    print("\nDownload from: https://github.com/opencv/opencv/tree/master/samples/dnn/face_detector")
    FACE_MODEL_AVAILABLE = False

if FACE_MODEL_AVAILABLE:
    # Load face detection model
    face_net = load_model_safe('caffe', str(face_model), str(face_prototxt))
    
    if face_net is not None:
        # Set default backend and target
        face_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        face_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
        print("\n✓ Face detection model ready")
        inspect_network(face_net, verbose=False)

### DNN 人臉檢測實作

In [ ]:
def detect_faces_dnn(image, net, confidence_threshold=0.5):
    """
    Detect faces using DNN model
    
    Parameters:
    -----------
    image : np.ndarray
        Input image (BGR)
    net : cv2.dnn.Net
        Face detection network
    confidence_threshold : float
        Minimum confidence for detection (0.0-1.0)
        
    Returns:
    --------
    detections : list
        List of (x, y, w, h, confidence) tuples
    elapsed : float
        Detection time in seconds
    """
    h, w = image.shape[:2]
    
    # Prepare blob from image
    # Mean values from ImageNet dataset
    blob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)),
        scalefactor=1.0,
        size=(300, 300),
        mean=(104.0, 177.0, 123.0),
        swapRB=False,
        crop=False
    )
    
    # Forward pass
    start_time = time.time()
    net.setInput(blob)
    detections_raw = net.forward()
    elapsed = time.time() - start_time
    
    # Parse detections
    detections = []
    
    for i in range(detections_raw.shape[2]):
        confidence = detections_raw[0, 0, i, 2]
        
        if confidence > confidence_threshold:
            # Get bounding box coordinates
            box = detections_raw[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")
            
            # Convert to (x, y, w, h) format
            x = max(0, x1)
            y = max(0, y1)
            width = min(w - x, x2 - x1)
            height = min(h - y, y2 - y1)
            
            detections.append((x, y, width, height, confidence))
    
    return detections, elapsed


def draw_faces_with_confidence(image, detections, color=(0, 255, 0), thickness=2):
    """
    Draw face bounding boxes with confidence scores
    
    Parameters:
    -----------
    image : np.ndarray
        Input image
    detections : list
        List of (x, y, w, h, confidence) tuples
    color : tuple
        Box color (B, G, R)
    thickness : int
        Line thickness
        
    Returns:
    --------
    output : np.ndarray
        Image with drawn boxes
    """
    output = image.copy()
    
    for i, (x, y, w, h, conf) in enumerate(detections):
        # Draw rectangle
        cv2.rectangle(output, (x, y), (x + w, y + h), color, thickness)
        
        # Draw label with confidence
        label = f"Face {i+1}: {conf:.2f}"
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        
        # Background for text
        cv2.rectangle(output, 
                     (x, y - label_size[1] - 10), 
                     (x + label_size[0], y), 
                     color, -1)
        
        # Text
        cv2.putText(output, label, (x, y - 5), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    return output

print("Face detection functions defined.")

### 測試人臉檢測

In [ ]:
if FACE_MODEL_AVAILABLE and face_net is not None:
    # Load test image
    test_image_paths = [
        '../assets/images/basic/baby.jpg',
        '../assets/images/basic/baby01.jpg',
        '../assets/images/faces/face01.jpg'
    ]
    
    # Find first existing image
    test_img = None
    for path in test_image_paths:
        if Path(path).exists():
            test_img = cv2.imread(path)
            print(f"Loaded test image: {path}")
            break
    
    if test_img is None:
        # Create synthetic image
        test_img = np.ones((480, 640, 3), dtype=np.uint8) * 200
        cv2.putText(test_img, "No test image found", (150, 240),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
        print("Created synthetic test image")
    
    # Detect faces
    detections, elapsed = detect_faces_dnn(test_img, face_net, confidence_threshold=0.5)
    
    # Draw results
    result_img = draw_faces_with_confidence(test_img, detections)
    
    # Display
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))
    
    axes[0].imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original Image', fontsize=12)
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'DNN Face Detection\n'
                     f'Detected: {len(detections)} faces, '
                     f'Time: {elapsed*1000:.2f}ms',
                     fontsize=12)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print details
    print(f"\nDetection Results:")
    print(f"  Faces detected: {len(detections)}")
    print(f"  Inference time: {elapsed*1000:.2f}ms")
    print(f"\nDetection details:")
    for i, (x, y, w, h, conf) in enumerate(detections, 1):
        print(f"  Face {i}: ({x}, {y}, {w}, {h}), confidence={conf:.3f}")
        
else:
    print("Face detection model not available. Skipping demo.")

### Haar Cascade vs DNN 比較

In [ ]:
if FACE_MODEL_AVAILABLE and face_net is not None and test_img is not None:
    # Load Haar Cascade for comparison
    haar_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    )
    
    # Haar detection
    gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    start_time = time.time()
    haar_faces = haar_cascade.detectMultiScale(gray, 1.1, 5, minSize=(30, 30))
    haar_time = time.time() - start_time
    
    # Draw Haar results
    haar_result = test_img.copy()
    for (x, y, w, h) in haar_faces:
        cv2.rectangle(haar_result, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Compare
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))
    
    axes[0].imshow(cv2.cvtColor(haar_result, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f'Haar Cascade\n'
                     f'Detected: {len(haar_faces)} faces, '
                     f'Time: {haar_time*1000:.2f}ms',
                     fontsize=12)
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'DNN (ResNet-SSD)\n'
                     f'Detected: {len(detections)} faces, '
                     f'Time: {elapsed*1000:.2f}ms',
                     fontsize=12)
    axes[1].axis('off')
    
    plt.suptitle('Face Detection Comparison', fontsize=14, y=0.98)
    plt.tight_layout()
    plt.show()
    
    # Comparison table
    print("\nMethod Comparison:")
    print("=" * 60)
    print(f"{'Method':<20} {'Faces':<10} {'Time (ms)':<15} {'Accuracy'}")
    print("=" * 60)
    print(f"{'Haar Cascade':<20} {len(haar_faces):<10} {haar_time*1000:<15.2f} {'Medium'}")
    print(f"{'DNN (ResNet-SSD)':<20} {len(detections):<10} {elapsed*1000:<15.2f} {'High'}")
    print("=" * 60)
    
    print("\nKey differences:")
    print("  - DNN provides confidence scores for each detection")
    print("  - DNN handles side faces and occlusions better")
    print("  - DNN has fewer false positives")
    print("  - Haar is faster but less accurate")
else:
    print("Comparison skipped: model or image not available")

## 5. 物體檢測 (Object Detection) - 20%

### MobileNet-SSD 物體檢測

MobileNet-SSD 是一個輕量級的物體檢測模型，適合實時應用。

### 模型特點
- **架構**: Single Shot Detector (SSD) with MobileNet backbone
- **輸入**: 300×300 RGB image
- **類別**: 20 object classes (PASCAL VOC)
- **優勢**: 快速、輕量、適合移動設備

In [ ]:
# MobileNet-SSD class labels (PASCAL VOC)
CLASSES = [
    "background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair",
    "cow", "diningtable", "dog", "horse", "motorbike",
    "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

# Color map for visualization
np.random.seed(42)
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

print(f"MobileNet-SSD can detect {len(CLASSES) - 1} object classes:")
for i, cls in enumerate(CLASSES[1:], 1):
    print(f"  {i:2d}. {cls}")

# Setup object detection model paths
OBJECT_MODEL_DIR = Path('../assets/models/object_detection')
OBJECT_MODEL_DIR.mkdir(parents=True, exist_ok=True)

mobilenet_prototxt = OBJECT_MODEL_DIR / 'MobileNetSSD_deploy.prototxt'
mobilenet_model = OBJECT_MODEL_DIR / 'MobileNetSSD_deploy.caffemodel'

# Check if models exist
if mobilenet_prototxt.exists() and mobilenet_model.exists():
    print("\n✓ MobileNet-SSD models found")
    OBJECT_MODEL_AVAILABLE = True
else:
    print("\n⚠ MobileNet-SSD models not found")
    print("Download from: https://github.com/chuanqi305/MobileNet-SSD")
    print(f"Save to: {OBJECT_MODEL_DIR}")
    OBJECT_MODEL_AVAILABLE = False

if OBJECT_MODEL_AVAILABLE:
    # Load MobileNet-SSD model
    object_net = load_model_safe('caffe', str(mobilenet_model), str(mobilenet_prototxt))
    
    if object_net is not None:
        object_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        object_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
        print("\n✓ Object detection model ready")

### 物體檢測實作

In [ ]:
def detect_objects(image, net, confidence_threshold=0.2):
    """
    Detect objects using MobileNet-SSD
    
    Parameters:
    -----------
    image : np.ndarray
        Input image (BGR)
    net : cv2.dnn.Net
        Object detection network
    confidence_threshold : float
        Minimum confidence for detection
        
    Returns:
    --------
    detections : list
        List of (class_id, class_name, confidence, x, y, w, h) tuples
    elapsed : float
        Detection time
    """
    h, w = image.shape[:2]
    
    # Prepare blob
    blob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)),
        scalefactor=0.007843,  # 1/127.5
        size=(300, 300),
        mean=127.5,
        swapRB=True,
        crop=False
    )
    
    # Forward pass
    start_time = time.time()
    net.setInput(blob)
    detections_raw = net.forward()
    elapsed = time.time() - start_time
    
    # Parse detections
    detections = []
    
    for i in range(detections_raw.shape[2]):
        confidence = detections_raw[0, 0, i, 2]
        
        if confidence > confidence_threshold:
            # Get class ID
            class_id = int(detections_raw[0, 0, i, 1])
            
            # Get bounding box
            box = detections_raw[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")
            
            x = max(0, x1)
            y = max(0, y1)
            width = min(w - x, x2 - x1)
            height = min(h - y, y2 - y1)
            
            detections.append((
                class_id,
                CLASSES[class_id],
                confidence,
                x, y, width, height
            ))
    
    return detections, elapsed


def draw_detections(image, detections):
    """
    Draw object detection results
    
    Parameters:
    -----------
    image : np.ndarray
        Input image
    detections : list
        List of detection tuples
        
    Returns:
    --------
    output : np.ndarray
        Image with drawn boxes and labels
    """
    output = image.copy()
    
    for (class_id, class_name, confidence, x, y, w, h) in detections:
        # Get color
        color = COLORS[class_id].astype(int).tolist()
        
        # Draw box
        cv2.rectangle(output, (x, y), (x + w, y + h), color, 2)
        
        # Prepare label
        label = f"{class_name}: {confidence:.2f}"
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        
        # Draw label background
        cv2.rectangle(output,
                     (x, y - label_size[1] - 10),
                     (x + label_size[0], y),
                     color, -1)
        
        # Draw label text
        cv2.putText(output, label, (x, y - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    return output

print("Object detection functions defined.")

### 測試物體檢測

In [ ]:
if OBJECT_MODEL_AVAILABLE and object_net is not None:
    # Load test image with objects
    test_paths = [
        '../assets/images/basic/assassin.jpg',
        '../assets/images/objects/car.jpg',
        '../assets/images/basic/1.jpg'
    ]
    
    test_img = None
    for path in test_paths:
        if Path(path).exists():
            test_img = cv2.imread(path)
            print(f"Loaded test image: {path}")
            break
    
    if test_img is None:
        # Create demo image
        test_img = np.ones((480, 640, 3), dtype=np.uint8) * 200
        cv2.putText(test_img, "Place test image here", (150, 240),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
        print("Created demo image")
    
    # Detect objects
    detections, elapsed = detect_objects(test_img, object_net, confidence_threshold=0.3)
    
    # Draw results
    result_img = draw_detections(test_img, detections)
    
    # Display
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    axes[0].imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original Image', fontsize=12)
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'MobileNet-SSD Detection\n'
                     f'Detected: {len(detections)} objects, '
                     f'Time: {elapsed*1000:.2f}ms',
                     fontsize=12)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print detection details
    print(f"\nDetection Results:")
    print(f"  Objects detected: {len(detections)}")
    print(f"  Inference time: {elapsed*1000:.2f}ms")
    print(f"\nDetection details:")
    for i, (cid, cname, conf, x, y, w, h) in enumerate(detections, 1):
        print(f"  {i}. {cname}: confidence={conf:.3f}, bbox=({x}, {y}, {w}, {h})")
        
else:
    print("Object detection model not available. Skipping demo.")

### 批次圖像檢測

In [ ]:
if OBJECT_MODEL_AVAILABLE and object_net is not None:
    # Find multiple test images
    image_dir = Path('../assets/images/basic')
    
    if image_dir.exists():
        image_files = list(image_dir.glob('*.jpg'))[:4]
        
        if len(image_files) > 0:
            fig, axes = plt.subplots(2, 2, figsize=(16, 12))
            axes = axes.ravel()
            
            for idx, img_path in enumerate(image_files[:4]):
                # Load image
                img = cv2.imread(str(img_path))
                if img is None:
                    continue
                
                # Resize for display
                img = cv2.resize(img, (640, 480))
                
                # Detect
                dets, t = detect_objects(img, object_net, confidence_threshold=0.4)
                
                # Draw
                result = draw_detections(img, dets)
                
                # Display
                axes[idx].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
                axes[idx].set_title(f'{img_path.name}\n'
                                   f'{len(dets)} objects, {t*1000:.1f}ms',
                                   fontsize=10)
                axes[idx].axis('off')
            
            plt.suptitle('Batch Object Detection', fontsize=14, y=0.98)
            plt.tight_layout()
            plt.show()
        else:
            print("No test images found")
    else:
        print(f"Image directory not found: {image_dir}")
else:
    print("Skipping batch detection demo")

## 6. 圖像分類 (Image Classification) - 10%

### 使用 GoogLeNet 進行圖像分類

GoogLeNet (Inception v1) 是一個經典的圖像分類網絡，在 ImageNet 數據集上訓練。

### 模型特點
- **架構**: GoogLeNet (Inception v1)
- **輸入**: 224×224 RGB image
- **類別**: 1000 ImageNet classes
- **應用**: 通用圖像分類

In [ ]:
# Note: Due to file size, classification models are typically not included
# This is a demonstration of how to use them

print("Image Classification with DNN")
print("=" * 60)
print("\nPopular classification models:")
print("  1. GoogLeNet (Inception v1) - 7MB")
print("  2. ResNet-50 - 100MB")
print("  3. VGG-16 - 550MB")
print("  4. SqueezeNet - 5MB (lightweight)")
print("  5. MobileNet - 17MB (mobile-friendly)")
print("\nAll models can classify 1000 ImageNet categories")

# Classification function template
def classify_image(image, net, labels, top_k=5):
    """
    Classify image using pre-trained model
    
    Parameters:
    -----------
    image : np.ndarray
        Input image
    net : cv2.dnn.Net
        Classification network
    labels : list
        List of class labels
    top_k : int
        Return top K predictions
        
    Returns:
    --------
    predictions : list
        List of (class_name, probability) tuples
    elapsed : float
        Inference time
    """
    # Prepare blob (ImageNet mean values)
    blob = cv2.dnn.blobFromImage(
        image,
        scalefactor=1.0,
        size=(224, 224),
        mean=(104, 117, 123),
        swapRB=False,
        crop=False
    )
    
    # Forward pass
    start_time = time.time()
    net.setInput(blob)
    probs = net.forward()
    elapsed = time.time() - start_time
    
    # Get top K predictions
    probs = probs.flatten()
    top_indices = probs.argsort()[-top_k:][::-1]
    
    predictions = []
    for idx in top_indices:
        predictions.append((labels[idx], probs[idx]))
    
    return predictions, elapsed

print("\n✓ Classification function template defined")
print("\nTo use classification:")
print("  1. Download a model (e.g., GoogLeNet from Caffe Model Zoo)")
print("  2. Download ImageNet class labels")
print("  3. Load model: net = cv2.dnn.readNetFromCaffe(prototxt, model)")
print("  4. Classify: predictions = classify_image(img, net, labels)")

## 7. 語義分割 (Semantic Segmentation) - 10%

### 使用 ENet 進行語義分割

語義分割為圖像的每個像素分配類別標籤。

### 分割模型類型
1. **FCN (Fully Convolutional Network)** - 第一個端到端分割網絡
2. **ENet** - 輕量級實時分割網絡
3. **Mask R-CNN** - 實例分割 (結合檢測與分割)
4. **DeepLab v3+** - 高精度語義分割

In [ ]:
print("Semantic Segmentation with DNN")
print("=" * 60)
print("\nCommon segmentation tasks:")
print("  1. Semantic Segmentation - Classify each pixel")
print("  2. Instance Segmentation - Separate object instances")
print("  3. Panoptic Segmentation - Combine semantic + instance")

# Segmentation function template
def segment_image(image, net, output_layer_name):
    """
    Perform semantic segmentation
    
    Parameters:
    -----------
    image : np.ndarray
        Input image
    net : cv2.dnn.Net
        Segmentation network
    output_layer_name : str
        Name of output layer
        
    Returns:
    --------
    mask : np.ndarray
        Segmentation mask (H, W) with class IDs
    elapsed : float
        Inference time
    """
    h, w = image.shape[:2]
    
    # Prepare blob
    blob = cv2.dnn.blobFromImage(
        image,
        scalefactor=1.0/255.0,
        size=(512, 512),
        mean=(0, 0, 0),
        swapRB=True,
        crop=False
    )
    
    # Forward pass
    start_time = time.time()
    net.setInput(blob)
    output = net.forward(output_layer_name)
    elapsed = time.time() - start_time
    
    # Output shape: (1, num_classes, H, W)
    # Get class with max probability for each pixel
    mask = np.argmax(output[0], axis=0)
    
    # Resize to original size
    mask = cv2.resize(mask.astype(np.uint8), (w, h), interpolation=cv2.INTER_NEAREST)
    
    return mask, elapsed


def visualize_segmentation(image, mask, num_classes=21, alpha=0.6):
    """
    Visualize segmentation mask
    
    Parameters:
    -----------
    image : np.ndarray
        Original image
    mask : np.ndarray
        Segmentation mask with class IDs
    num_classes : int
        Number of classes
    alpha : float
        Transparency of overlay
        
    Returns:
    --------
    output : np.ndarray
        Visualization
    """
    # Create color map
    colors = np.random.randint(0, 255, size=(num_classes, 3), dtype=np.uint8)
    colors[0] = [0, 0, 0]  # Background is black
    
    # Map mask to colors
    colored_mask = colors[mask]
    
    # Blend with original image
    output = cv2.addWeighted(image, 1 - alpha, colored_mask, alpha, 0)
    
    return output

print("\n✓ Segmentation function templates defined")
print("\nPopular segmentation datasets:")
print("  - PASCAL VOC: 21 classes (person, car, dog, etc.)")
print("  - Cityscapes: Urban street scenes (19 classes)")
print("  - ADE20K: 150 diverse classes")
print("  - COCO: 80 object categories")

## 8. 性能優化 (Performance Optimization) - 10%

### Backend 與 Target 選擇

OpenCV DNN 支援多種計算後端和目標設備。

### Backends
- **DNN_BACKEND_DEFAULT**: 自動選擇
- **DNN_BACKEND_OPENCV**: OpenCV 實作 (CPU)
- **DNN_BACKEND_HALIDE**: Halide 優化
- **DNN_BACKEND_CUDA**: NVIDIA CUDA (GPU)
- **DNN_BACKEND_INFERENCE_ENGINE**: Intel OpenVINO

### Targets
- **DNN_TARGET_CPU**: CPU 執行
- **DNN_TARGET_OPENCL**: OpenCL (GPU/CPU)
- **DNN_TARGET_OPENCL_FP16**: OpenCL with FP16
- **DNN_TARGET_CUDA**: CUDA (NVIDIA GPU)
- **DNN_TARGET_CUDA_FP16**: CUDA with FP16
- **DNN_TARGET_MYRIAD**: Intel Movidius VPU

In [ ]:
# Performance benchmark utility
def benchmark_backend_target(net, image, backend, target, iterations=10):
    """
    Benchmark DNN inference with specific backend and target
    
    Parameters:
    -----------
    net : cv2.dnn.Net
        Network to benchmark
    image : np.ndarray
        Test image
    backend : int
        DNN backend ID
    target : int
        DNN target ID
    iterations : int
        Number of iterations for averaging
        
    Returns:
    --------
    avg_time : float
        Average inference time in milliseconds
    success : bool
        Whether benchmark succeeded
    """
    try:
        # Set backend and target
        net.setPreferableBackend(backend)
        net.setPreferableTarget(target)
        
        # Prepare blob
        blob = cv2.dnn.blobFromImage(
            cv2.resize(image, (300, 300)),
            1.0, (300, 300), (104, 177, 123),
            swapRB=False, crop=False
        )
        
        # Warmup
        net.setInput(blob)
        _ = net.forward()
        
        # Benchmark
        times = []
        for _ in range(iterations):
            start = time.time()
            net.setInput(blob)
            _ = net.forward()
            elapsed = time.time() - start
            times.append(elapsed * 1000)  # Convert to ms
        
        avg_time = np.mean(times)
        return avg_time, True
        
    except Exception as e:
        print(f"    Error: {e}")
        return 0, False

print("Performance benchmarking utility defined.")

### 性能比較測試

In [ ]:
if FACE_MODEL_AVAILABLE and face_net is not None and test_img is not None:
    print("Benchmarking different backend/target combinations...\n")
    
    # Test configurations
    configs = [
        ("CPU", cv2.dnn.DNN_BACKEND_OPENCV, cv2.dnn.DNN_TARGET_CPU),
        ("OpenCL", cv2.dnn.DNN_BACKEND_OPENCV, cv2.dnn.DNN_TARGET_OPENCL),
        ("OpenCL FP16", cv2.dnn.DNN_BACKEND_OPENCV, cv2.dnn.DNN_TARGET_OPENCL_FP16),
    ]
    
    # Add CUDA if available
    try:
        if cv2.cuda.getCudaEnabledDeviceCount() > 0:
            configs.append(("CUDA", cv2.dnn.DNN_BACKEND_CUDA, cv2.dnn.DNN_TARGET_CUDA))
            configs.append(("CUDA FP16", cv2.dnn.DNN_BACKEND_CUDA, cv2.dnn.DNN_TARGET_CUDA_FP16))
    except:
        pass
    
    results = []
    
    for name, backend, target in configs:
        print(f"Testing {name}...", end=" ")
        avg_time, success = benchmark_backend_target(
            face_net, test_img, backend, target, iterations=10
        )
        
        if success:
            print(f"✓ {avg_time:.2f}ms")
            results.append((name, avg_time))
        else:
            print("✗ Not available")
    
    # Visualize results
    if len(results) > 0:
        names, times = zip(*results)
        
        fig, ax = plt.subplots(figsize=(10, 6))
        bars = ax.barh(names, times, color='skyblue', edgecolor='black')
        
        # Color fastest bar green
        min_idx = np.argmin(times)
        bars[min_idx].set_color('lightgreen')
        
        # Add value labels
        for i, (name, time_val) in enumerate(results):
            ax.text(time_val + 0.5, i, f'{time_val:.2f}ms',
                   va='center', fontsize=10)
        
        ax.set_xlabel('Inference Time (ms)', fontsize=12)
        ax.set_title('Performance Comparison: Backend/Target', fontsize=14, pad=20)
        ax.grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Speedup analysis
        baseline_time = results[0][1]  # CPU time
        print("\nSpeedup compared to CPU:")
        for name, time_val in results:
            speedup = baseline_time / time_val
            print(f"  {name:<15}: {speedup:.2f}x")
else:
    print("Skipping performance benchmark (model not available)")

### 優化建議

#### 1. 選擇合適的 Backend/Target
```python
# CPU (default)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# NVIDIA GPU (if available)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

# Intel OpenCL
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)
```

#### 2. 模型量化 (Quantization)
- 使用 FP16 或 INT8 精度
- 減少模型大小和推理時間
- 略微降低精度（通常可接受）

#### 3. 批次處理 (Batching)
- 使用 `blobFromImages()` 處理多張圖像
- 提高 GPU 利用率

#### 4. 輸入尺寸優化
- 使用較小的輸入尺寸（如 300×300 vs 600×600）
- 權衡精度與速度

#### 5. 模型選擇
- MobileNet: 移動設備
- SqueezeNet: 邊緣設備
- ResNet/VGG: 高精度應用

In [ ]:
# Optimization comparison table
print("\nOptimization Strategies:")
print("=" * 80)
print(f"{'Strategy':<25} {'Speedup':<15} {'Accuracy Impact':<20} {'Use Case'}")
print("=" * 80)
print(f"{'FP16 Precision':<25} {'1.5-2x':<15} {'Minimal (~0.5%)':<20} {'GPU inference'}")
print(f"{'INT8 Quantization':<25} {'2-4x':<15} {'Small (~1-2%)':<20} {'Edge devices'}")
print(f"{'Model Pruning':<25} {'1.5-3x':<15} {'Medium (~2-5%)':<20} {'Resource-limited'}")
print(f"{'Input Downscaling':<25} {'2-4x':<15} {'Depends on task':<20} {'Real-time apps'}")
print(f"{'Batch Processing':<25} {'1.5-2x':<15} {'None':<20} {'Offline processing'}")
print(f"{'GPU Acceleration':<25} {'5-10x':<15} {'None':<20} {'Server deployment'}")
print("=" * 80)

## 9. 實戰練習 (Hands-on Exercises) - 3%

### 練習 1: 多模型整合

結合人臉檢測和物體檢測，在同一張圖像上運行兩個模型。

**任務**:
1. 載入人臉檢測和物體檢測模型
2. 對同一圖像進行兩種檢測
3. 在結果圖上用不同顏色標記
4. 統計檢測到的人臉和物體數量

### 練習 2: 視訊流處理

實作實時視訊物體檢測。

**任務**:
1. 讀取視訊文件或攝像頭
2. 逐幀進行物體檢測
3. 計算並顯示 FPS
4. 測試不同 backend 的性能差異

### 練習 3: 自定義後處理

實作 Non-Maximum Suppression (NMS) 去除重複檢測。

**任務**:
1. 實作 IoU (Intersection over Union) 計算
2. 實作 NMS 算法
3. 對檢測結果應用 NMS
4. 比較 NMS 前後的結果

In [ ]:
# Exercise templates

# Exercise 1: Multi-model integration
def exercise1_multimodel():
    """
    TODO: Implement multi-model detection
    - Load face and object detection models
    - Run both on same image
    - Draw results with different colors
    - Print statistics
    """
    pass

# Exercise 2: Video stream processing
def exercise2_video():
    """
    TODO: Implement real-time video detection
    - Open video file or webcam
    - Process each frame
    - Calculate and display FPS
    - Test different backends
    """
    pass

# Exercise 3: NMS implementation
def compute_iou(box1, box2):
    """
    TODO: Compute Intersection over Union
    
    Parameters:
    -----------
    box1, box2 : tuple
        Bounding boxes (x, y, w, h)
        
    Returns:
    --------
    iou : float
        IoU score (0.0-1.0)
    """
    pass

def apply_nms(detections, iou_threshold=0.5):
    """
    TODO: Apply Non-Maximum Suppression
    
    Parameters:
    -----------
    detections : list
        List of (class_id, confidence, x, y, w, h) tuples
    iou_threshold : float
        IoU threshold for suppression
        
    Returns:
    --------
    filtered : list
        Filtered detections after NMS
    """
    pass

print("Exercise templates defined.")
print("\nImplement the TODO sections to complete the exercises.")

## 10. 總結與延伸 (Summary & Extensions) - 2%

### 關鍵要點

1. **OpenCV DNN 優勢**
   - 統一接口支援多框架模型
   - 無需安裝深度學習框架
   - 輕量級、易部署
   - 支援多種硬體加速

2. **支援的框架**
   - Caffe: 人臉檢測、分類
   - TensorFlow: 物體檢測
   - PyTorch (via ONNX): 最新研究模型
   - Darknet: YOLO 系列

3. **應用場景**
   - 人臉檢測: ResNet-SSD
   - 物體檢測: MobileNet-SSD, YOLO
   - 圖像分類: GoogLeNet, ResNet
   - 語義分割: FCN, ENet

4. **性能優化**
   - Backend/Target 選擇
   - FP16/INT8 量化
   - 批次處理
   - 輸入尺寸優化

### 與傳統方法對比

| 方法 | 準確度 | 速度 | 部署 | 適用場景 |
|------|--------|------|------|----------|
| **Haar Cascade** | 中 (85%) | 快 (10ms) | 簡單 | 實時應用、嵌入式 |
| **HOG + SVM** | 中-高 (90%) | 中 (50ms) | 簡單 | 傳統 CV 系統 |
| **DNN (CPU)** | 高 (95%+) | 中 (30-80ms) | 中等 | 準確度優先 |
| **DNN (GPU)** | 高 (95%+) | 快 (5-15ms) | 複雜 | 實時 + 高精度 |

### 延伸學習

#### 1. 模型訓練與轉換
- 使用 TensorFlow/PyTorch 訓練自定義模型
- 轉換為 OpenCV DNN 支援格式
- 模型量化與優化

#### 2. YOLO 系列
- YOLOv3/v4/v5 實戰
- 實時多類別物體檢測
- 自定義類別訓練

#### 3. 實例分割
- Mask R-CNN 實作
- 像素級物體分割
- 醫療影像應用

#### 4. 跨平台部署
- Android/iOS 移動部署
- 邊緣設備優化 (Raspberry Pi, Jetson)
- TensorFlow Lite / ONNX Runtime

#### 5. 深度學習框架整合
- OpenCV + TensorFlow
- OpenCV + PyTorch
- 混合推理管線

### 實戰項目建議

1. **智能監控系統**
   - 多攝像頭人臉 + 物體檢測
   - 異常行為檢測
   - 實時警報系統

2. **自動駕駛感知**
   - 車輛、行人、交通標誌檢測
   - 車道線分割
   - 多傳感器融合

3. **醫療影像分析**
   - 病灶檢測與分割
   - X光/CT/MRI 分析
   - 輔助診斷系統

4. **零售分析**
   - 商品識別
   - 顧客行為分析
   - 庫存管理

### 參考資源

#### 官方文檔
- OpenCV DNN Tutorial: https://docs.opencv.org/4.x/d2/d58/tutorial_table_of_content_dnn.html
- OpenCV Model Zoo: https://github.com/opencv/opencv/wiki/Deep-Learning-in-OpenCV

#### 模型資源
- Caffe Model Zoo: https://github.com/BVLC/caffe/wiki/Model-Zoo
- TensorFlow Model Garden: https://github.com/tensorflow/models
- ONNX Model Zoo: https://github.com/onnx/models
- PyTorch Hub: https://pytorch.org/hub/

#### 論文與教程
- SSD: Single Shot MultiBox Detector (2016)
- YOLO: You Only Look Once (2015-2020)
- Mask R-CNN (2017)
- EfficientDet (2020)

---

## 下一步

完成本模組後，建議繼續學習:
- **5.2.2 YOLO 系列實戰** - YOLOv3/v4/v5 詳細教學
- **5.2.3 模型訓練與轉換** - 自定義模型開發
- **5.3.1 實例分割** - Mask R-CNN 實作
- **5.3.2 姿態估計** - OpenPose 人體骨架檢測

**實戰建議**: 
1. 下載並測試不同模型
2. 比較各模型的速度與精度
3. 實作自己的檢測應用
4. 嘗試模型融合提升性能

---

**模組完成標記**: ✅ WBS 5.2.1 OpenCV DNN Module Complete